In [73]:
import pandas as pd

In [74]:
import numpy as np

In [127]:
import dynamic_portfolio as dp
import dynamic_portfolio.cross_validate as cv

# Loading data and creating clean dataframe

In [ ]:
df = features_creation('META')

In [ ]:
pd.set_option('display.max_columns', None)

# Baseline

In [ ]:
backtest_mask = final_df['date'] > '2020-01-02'

In [ ]:
backtest_df = final_df[backtest_mask]

In [ ]:
model_mask = final_df['date'] <= '2020-01-02'

In [ ]:
model_df = final_df[model_mask]

In [ ]:
model_df[['return']].iloc[-1]

In [ ]:
backtest_df[['return']].iloc[0]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(backtest_df[['return']].iloc[0],
                   model_df[['return']].iloc[-1])**0.5

In [ ]:
def baseline_model(df):
    df = y_train
    y_pred = y_train[-1]
    return y_pred

In [ ]:
def rmse():
    rmse = (np.mean((y_test[0] - y_pred)**2))**0.5
    return rmse

In [ ]:
X = model_df.drop(columns=['date'])

In [ ]:
y = model_df['return']

In [ ]:
model = model.fit(X, y)

In [ ]:
backtest_df = backtest_df.drop(columns=['date'])

In [ ]:
y_true = backtest_df['return']

In [ ]:
y_true

In [ ]:
backtest_df.iloc[[3]]

In [ ]:
backtest_df

In [ ]:
y_pred = []
backtest_df.reset_index(drop=True, inplace=True)
for i in range(len(backtest_df)):
    pred = model.predict(backtest_df.iloc[[i]])
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
y_pred[:, 0].shape

In [ ]:
y_true.shape

In [ ]:
rmse = (np.mean((y_true - y_pred[:, 0])**2))**0.5

In [ ]:
rmse

# Cross vals

We use sklearn's time series split to break up the data in different folds
We use a sklearn example to run the first tests.

Metrics used : 
 - rmse
 - mae
 - R2

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
ts_cv = TimeSeriesSplit(
    n_splits=20,
    gap=0,
    max_train_size=252,
    test_size=45,
)

In [76]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [5]:
def baseline_model():
    

SyntaxError: incomplete input (671652115.py, line 2)

In [6]:
model = HistGradientBoostingRegressor()

In [ ]:
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.6f} +/- {mae.std():.6f}\n"
        f"Root Mean Squared Error: {rmse.mean():.6f} +/- {rmse.std():.6f}"
    )


In [ ]:
import seaborn as sns

In [ ]:
evaluate(model, X, y, ts_cv)

In [ ]:
scores = cross_val_score(model, X, y, cv = ts_cv)

In [ ]:
scores = np.mean(scores)
scores

# PCA Analysis

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 0.95)
X_pca = pca.fit_transform(aapl) 
print(pca.n_components_) 

In [ ]:
pca.n_components

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(np.unique(pd.DataFrame(pca.components_, columns = aapl.columns).columns))


In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
n_pcs= pca.n_components_ 
# get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = aapl.columns
# get the most important feature names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

In [ ]:
n_pcs

In [ ]:
len(np.unique(most_important_names))

In [ ]:
np.unique(most_important_names)

In [64]:
from dynamic_portfolio.cross_validate import *

In [65]:
from sklearn.metrics import mean_squared_error

In [134]:
fold_length = 125 #
fold_stride = 60 #1 trimestre
train_test_ratio = 0.7
input_length = 0
horizon = 1
output_length = 1
#stride = 1

In [135]:
def get_folds(
    df: pd.DataFrame,
    fold_length: int,
    fold_stride: int):
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold

    Returns a list of folds, each as a DataFrame
    '''

    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds

In [98]:
df = ready_to_train_df('AAPL')

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [103]:
len(train_test_split(df, 0.7, 15,1)[0])+ len(train_test_split(df, 0.7, 15,1)[1])

4362

In [136]:
def train_test_split(fold: pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int,
                     horizon: int) :
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    '''

    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]

    # TEST SET
    # ======================
    first_test_idx = last_train_idx - input_length
    fold_test = fold.iloc[first_test_idx:, :]

    return (fold_train, fold_test)



In [137]:
def cross_validate_ml(df) :
    '''
    get_folds() create many FOLDS, train_test_split() create a split on ONE FOLDS.
    The goal of this function is to make splits and sequences on each FOLDS.
    Then, apply a model.
    '''
    folds = get_folds(df, fold_length, fold_stride) # 1 - Creating FOLDS
    rmses_model = []
    rmses_baseline = []
    
    for fold_id, fold in enumerate(folds):

        # 2 - CHRONOLOGICAL TRAIN TEST SPLIT of the current FOLD

        (fold_train, fold_test) = train_test_split(fold = fold,
                                                train_test_ratio=train_test_ratio,
                                                input_length = input_length,
                                                horizon=horizon)

        # 3 - Scanninng fold_train and fold_test for SEQUENCES

        X_train, y_train = fold_train, fold_train['return']

        X_test, y_test = fold_test, fold_test['return']

        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        rmse_model = (mean_squared_error(y_test, model.predict(X_test)))**0.5
        rmses_model.append(rmse_model)
        
        #print(type(y_train))
        #rmse_baseline = mean_squared_error(y_test.iloc[[0]], y_train.iloc[[-1]])**0.5
        #rmses_baseline.append(rmse_baseline)
        
    
        
        
        
    return np.mean(rmses_model)
        
        

In [42]:
model, baseline = cross_validate_ml(df)[0], cross_validate_ml(df)[1]

In [81]:
cross_validate_ml(df)

0.008603253748945635

In [82]:
df

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,...,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,-0.998390,0.0,0.0,0.448416,0.0,0.801740,-0.569891,1.133770,0.171681,-0.317363,...,-0.419313,1.187464,0.743017,0.301611,-0.068194,1.718868,0.0,1.940652,-0.456106,0.875470
1,-1.032198,0.0,0.0,0.448416,0.0,0.603962,-0.609053,1.030990,0.148834,-0.317363,...,0.723640,-1.392291,-1.117542,-0.365681,-0.081601,-0.770408,0.0,1.940652,-0.456106,0.875470
2,-1.024241,0.0,0.0,0.448416,0.0,0.693861,-0.569891,1.056685,0.177181,-0.317363,...,-0.033843,0.666983,1.147240,0.075767,0.141698,-0.637953,0.0,1.940652,-0.456106,0.875470
3,-1.008716,0.0,0.0,0.448416,0.0,0.891639,-0.504622,1.133770,0.160258,-0.317363,...,-0.915490,1.433922,1.812472,0.255088,-0.055741,0.199758,0.0,1.940652,-0.456106,0.875470
4,-1.006878,0.0,0.0,0.448416,0.0,0.963558,-0.491568,1.172313,0.251221,-0.317363,...,0.174107,0.504010,0.315338,0.116275,0.414262,0.064280,0.0,1.940652,-0.456106,0.875470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,0.306056,0.0,0.0,0.702970,0.0,-2.452599,-0.961507,-0.806206,-1.310391,-0.312340,...,-1.523843,3.797413,2.988333,0.902844,0.775337,0.584562,0.0,0.740619,5.821427,-8.921692
2430,1.431119,0.0,0.0,0.702970,0.0,-2.506539,-0.961507,-0.844749,-1.366238,-0.312340,...,0.546709,-1.134041,-0.018786,-0.434260,-0.440184,-0.755108,0.0,0.740619,5.821427,-8.921692
2431,0.938401,0.0,0.0,0.702970,0.0,-2.578458,-0.987615,-0.870444,-1.331968,-0.312340,...,1.163206,-1.565516,-1.750158,-0.307262,0.308312,-0.711269,0.0,0.740619,5.821427,-8.921692
2432,2.207120,0.0,0.0,0.702970,0.0,-2.740276,-1.026776,-0.947529,-1.308276,-0.312340,...,1.279844,-3.697007,-2.875550,-0.918836,0.214426,-0.852736,0.0,0.740619,5.821427,-8.921692


In [ ]:
improvement = baseline/model
improvement

In [ ]:
baseline

In [43]:
model

0.00373410786557851

In [84]:
tickers = return_tickers()

In [93]:
tickers

['AAPL',
 'MSFT',
 'GOOG',
 'AMZN',
 'TSLA',
 'UNH',
 'XOM',
 'JNJ',
 'WMT',
 'NVDA',
 'JPM',
 'V',
 'CVX',
 'PG',
 'LLY',
 'MA',
 'HD',
 'META',
 'BAC',
 'ABBV',
 'PFE',
 'KO',
 'MRK',
 'PEP',
 'COST',
 'ORCL',
 'AVGO',
 'TMO',
 'MCD',
 'CSCO',
 'ACN',
 'DHR',
 'TMUS',
 'ABT',
 'WFC',
 'DIS',
 'LIN',
 'NEE',
 'BMY',
 'NKE',
 'VZ',
 'TXN',
 'UPS',
 'COP',
 'ADBE',
 'CMCSA',
 'CRM',
 'PM',
 'MS',
 'AMGN',
 'SCHW',
 'HON',
 'RTX',
 'QCOM',
 'T',
 'IBM',
 'DE',
 'CVS',
 'LOW',
 'GS',
 'UNP',
 'NFLX',
 'LMT',
 'CAT',
 'AMD',
 'INTC',
 'ELV',
 'SPGI',
 'AXP',
 'SBUX',
 'INTU',
 'BLK',
 'ADP',
 'GILD',
 'PLD',
 'MDT',
 'BA',
 'AMT',
 'CI',
 'GE',
 'TJX',
 'ISRG',
 'C',
 'AMAT',
 'PYPL',
 'MDLZ',
 'CB',
 'SYK',
 'ADI',
 'MMC',
 'EOG',
 'NOW',
 'VRTX',
 'MO',
 'NOC',
 'EL',
 'REGN',
 'PGR',
 'BKNG',
 'DUK',
 'TGT',
 'SLB',
 'SO',
 'MMM',
 'ITW',
 'ZTS',
 'GD',
 'APD',
 'HUM',
 'MRNA',
 'BDX',
 'CSX',
 'WM',
 'PNC',
 'HCA',
 'ETN',
 'USB',
 'FISV',
 'SHW',
 'OXY',
 'CL',
 'MU',
 'CME',
 'AON',


In [92]:
tickers.remove('DLRKEYS')

In [85]:
from sklearn.ensemble import RandomForestRegressor

In [96]:
apple = ready_to_train_df('AAPL')

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


In [114]:
csv  = !cd ../raw_data/eps && ls | grep .csv

/Users/ericvincent/code/dynamic_portfolio/notebooks


In [123]:
len(csv)

500

In [121]:
csv2 = !cd ../raw_data/stocks_return/ && ls | grep .csv

In [122]:
len(csv2)

499

In [94]:
rmses = []


for ticker in tickers:
    df = ready_to_train_df(ticker)
    rmse = cross_validate_ml(df)
    rmses.append(rmse)


/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

...

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split = second_split.dropna(inplace=True)
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  #final_df['return'][0]=0
/Users/ericvincent/code/dynamic_portfolio/dynamic_portfolio/utils.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secon

TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [145]:
np.mean(rmses)

0.0046477779040525425

In [139]:
tickers[320]

'HPE'

In [144]:
df = ready_to_train_df('BF.B')

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/eps/data_BF.B.csv'

In [143]:
df

,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,...,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,-0.718898,0.0,0.0,-0.602296,0.0,1.227750,0.747470,0.275947,-1.606435,-1.294269,...,0.285072,0.216056,0.623975,-0.165298,1.141770,0.877314,0.008204,0.367911,0.111749,1.740757
1,-0.732525,0.0,0.0,-0.602296,0.0,1.181043,0.719920,0.263899,-1.618651,-1.107031,...,-0.338905,-0.552303,-0.335695,-0.022892,-0.589434,-0.828274,10.243450,-0.252102,0.266866,1.754648
2,-0.736345,0.0,0.0,-0.602296,0.0,1.162360,0.685483,0.300042,-1.634683,-1.107031,...,-0.721023,-0.225826,-0.419727,0.096293,-0.776746,0.022679,0.008204,-0.252102,0.266866,1.754648
3,-0.718389,0.0,0.0,-0.602296,0.0,1.134336,0.685483,0.263899,-1.654915,-1.107031,...,-0.035086,-0.338128,-0.021826,-0.081217,-0.991763,0.685276,0.008204,-0.252102,0.266866,1.397007
4,-0.742522,0.0,0.0,-0.602296,0.0,1.162360,0.671709,0.324138,-1.634683,-1.107031,...,0.087807,0.332036,-0.183859,0.155971,0.968991,-0.411806,0.008204,-0.252102,0.266866,1.397007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4205,0.370328,0.0,0.0,2.277433,0.0,-0.388305,0.575285,-1.507113,0.015529,1.454439,...,0.781830,-0.004159,-0.192485,0.436957,0.287729,-1.116135,0.008204,-0.356875,0.227389,0.878090
4206,0.010093,0.0,0.0,2.277433,0.0,-0.435012,0.568397,-1.555304,0.037288,1.454439,...,0.401538,-0.882718,-0.107809,-0.787407,0.335769,-0.404860,0.008204,-0.356875,0.227389,0.878090
4207,-0.166579,0.0,0.0,2.277433,0.0,-0.406988,0.575285,-1.531209,0.068589,1.454439,...,0.379247,0.532323,0.064488,0.476065,0.488686,-1.309829,0.008204,-0.356875,0.227389,0.878090
4208,-0.379366,0.0,0.0,2.277433,0.0,-0.435012,0.568397,-1.555304,0.088821,1.454439,...,-0.080414,-0.534994,-0.107809,-0.423407,0.303708,-0.222310,0.008204,-0.356875,0.227389,0.878090


In [138]:
cross_validate_ml(epam)

0.0068757502181766295